## Lotart Dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_top = pd.read_csv('vectors_top.csv', header=None)
df_top.shape

(28172, 1001)

In [3]:
# to np
X = pd.DataFrame.as_matrix(df_top.loc[1:5000, 1:1000])
X.shape

(5000, 1000)

In [4]:
from sklearn.decomposition import PCA
pca = PCA(n_components=128)
Y = pca.fit_transform(X)

In [5]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2, random_state=0, n_iter=5000, perplexity=25)
T = model.fit_transform(Y)

In [6]:
from bokeh.layouts import row, column
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, BoxSelectTool, CustomJS, Rect, Div
from bokeh.io import output_notebook

In [7]:
#output_file("output.html")
output_notebook()

Loading BokehJS ...

In [10]:
df_top.loc[1, 0]

'12757272--r73359--t1476466310--sser3--cabeza-de-hombre-oil-on-paper-rufino-tamayo-normal.jpg'

In [13]:
x = T[:, 0]
y = T[:, 1]

n = ['http://localhost:5000/'+i for i in df_top.loc[1:5000, 0]]

In [14]:
source = ColumnDataSource(dict(x=x, y=y, n=n))
p1 = figure(tools='box_select,box_zoom,wheel_zoom,pan,reset', plot_width=400, plot_height=400)
p1.scatter(x='x',y='y', source=source)

hover = HoverTool()
hover = HoverTool(
        tooltips="""
           <img src="@n" height="75" width="75" />
        """
    )

# Finally add/enable the tool
p1.add_tools(hover)

jscode="""
    var data = source.data;
    var start = cb_obj.start;
    var end = cb_obj.end;
    data['%s'] = [start + (end - start) / 2];
    data['%s'] = [end - start];
    source.trigger('change');
"""

source_zoom = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': []})

p1.x_range.callback = CustomJS(
        args=dict(source=source_zoom), code=jscode % ('x', 'width'))
p1.y_range.callback = CustomJS(
        args=dict(source=source_zoom), code=jscode % ('y', 'height'))

p2 = figure(title='See Zoom Window Here', tools='', plot_width=400, plot_height=400)
p2.scatter(x='x',y='y', source=source)
rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1, line_color='black', fill_color='black')
p2.add_glyph(source_zoom, rect)


div = Div(width=800, height=200)
jscode_select  = """
        var inds = cb_obj.selected['1d'].indices;
        var d = cb_obj.data;
        var args = [];
        for (var i=0; i<inds.length; i++ ) {
            args.push('<img style="display: inline; margin:3px;" src="'+ d['n'][inds[i]] +'" height="75" width="75" />');
        }
        div.text = args.join("");
    """
source.callback = CustomJS(args=dict(div=div), code=jscode_select)

layout = column(row(p1, p2), div)

show(layout)
